In [2]:
import pandas as pd
import numpy as np


In [3]:
# Set JAVA_HOME environment variable
import os
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/jdk-11.jdk/Contents/Home"
print(os.environ.get('JAVA_HOME'))

/Library/Java/JavaVirtualMachines/jdk-11.jdk/Contents/Home


In [4]:
# Connect to MongoDB

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import mongosetupvars as msv
db_name = msv.DATABASE
collection_name = msv.COLLECTION
# clean host and build a correct URI (safe)
import re
host_raw = msv.HOST
# remove any leading/trailing spaces and drop any path or query part
host_clean = re.split(r'[/?]', host_raw)[0]

uri = f"mongodb+srv://{msv.USERNAME}:{msv.PASSWORD}@{host_clean}/{db_name}"
# add query params after the DB path
uri = uri + "?retryWrites=true&w=majority"

# create pymongo client and verify (optional)
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
# Spark imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
print(pyspark.__version__)

spark = SparkSession.builder. \
    appName("MongoDBIntegration"). \
    config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.13:10.2.0"). \
    getOrCreate()

print("spark.version:", spark.version)
print("spark.jars.packages:", spark.sparkContext.getConf().get("spark.jars.packages"))

3.5.4


25/11/23 21:25:16 WARN Utils: Your hostname, Shubhans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.192.163.75 instead (on interface en0)
25/11/23 21:25:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/shubhan/anaconda3/envs/data_discovery/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/shubhan/.ivy2/cache
The jars for the packages stored in: /Users/shubhan/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2ba7e27-3366-4019-87d7-9e23fde52c08;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.13;10.2.0 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 2064ms :: artifacts dl 10ms
	:: modules in use:
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongodb#bson-record-codec;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-core;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-sync;4.8.2 from central in [default]
	org.mongodb.spark#mongo-spark-conne

spark.version: 3.5.4
spark.jars.packages: org.mongodb.spark:mongo-spark-connector_2.13:10.2.0


In [6]:
try:
    df = spark.read.format("mongodb").option("uri", uri).option("database", db_name).option("collection", collection_name).load()
    df.printSchema()
    df.show(5)
except Exception as e:
    import traceback; traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/_v/ncwq304d761gb02f1n7x_2vr0000gn/T/ipykernel_8825/89167977.py", line 2, in <module>
    df = spark.read.format("mongodb").option("uri", uri).option("database", db_name).option("collection", collection_name).load()
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shubhan/anaconda3/envs/data_discovery/lib/python3.12/site-packages/pyspark/sql/readwriter.py", line 314, in load
    return self._df(self._jreader.load())
                    ^^^^^^^^^^^^^^^^^^^^
  File "/Users/shubhan/anaconda3/envs/data_discovery/lib/python3.12/site-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/Users/shubhan/anaconda3/envs/data_discovery/lib/python3.12/site-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^


In [7]:
import pyspark
print("pyspark:", pyspark.__version__)           # shows PySpark version (3.4.1)
print("spark.version:", spark.version)          # Spark runtime version
print("sc.version:", spark.sparkContext.version) 
# Scala version (may raise if not available)
try:
    print("scala:", spark._jvm.scala.util.Properties.versionString)
except Exception:
    try:
        print("scala number:", spark._jvm.scala.util.Properties.versionNumberString)
    except Exception as e:
        print("could not read scala version:", e)
# Show the connector package you're asking Spark to download
print("spark.jars.packages:", spark.sparkContext.getConf().get("spark.jars.packages"))

pyspark: 3.5.4
spark.version: 3.5.4
sc.version: 3.5.4
scala: <py4j.java_gateway.JavaMember object at 0x11fcf3d40>
spark.jars.packages: org.mongodb.spark:mongo-spark-connector_2.13:10.2.0


In [8]:
import socket, re
# extract host:port from msv.HOST or config_uri
host = None
port = 27017
if 'msv' in globals():
    host = msv.HOST
    # if msv.HOST contains a port like "host:27017", split it
    if ':' in host:
        h, p = host.split(':',1)
        host, port = h, int(re.sub(r'\D.*','',p))
print('testing TCP connect to', host, port)
try:
    s = socket.create_connection((host, port), timeout=5)
    s.close()
    print('TCP connection succeeded')
except Exception as e:
    print('TCP connection failed:', e)

testing TCP connect to cluster0.tqttmgs.mongodb.net/?appName=Cluster0 27017
TCP connection failed: [Errno 8] nodename nor servname provided, or not known


In [9]:
#print("uri repr:", repr(uri))
#print("config_uri repr:", repr(config_uri) if 'config_uri' in globals() else 'config_uri not defined')

In [10]:
# run this in the same notebook where client was created
try:
    print("pymongo client nodes:", client.nodes)   # set of (host, port)
except Exception as e:
    print("could not read client.nodes:", e)

pymongo client nodes: frozenset({('ac-lvli3cd-shard-00-01.tqttmgs.mongodb.net', 27017), ('ac-lvli3cd-shard-00-00.tqttmgs.mongodb.net', 27017), ('ac-lvli3cd-shard-00-02.tqttmgs.mongodb.net', 27017)})


In [11]:
import socket
socket.create_connection(("ac-lvli3cd-shard-00-01.tqttmgs.mongodb.net", 27017), timeout=5)

<socket.socket fd=106, family=2, type=1, proto=6, laddr=('10.192.163.75', 51062), raddr=('89.192.8.246', 27017)>

In [12]:
#print("host_clean:", host_clean)
#print("uri:", uri.replace(msv.PASSWORD, '*****'))

In [13]:
print("pyspark:", __import__("pyspark").__version__)
print("spark.version:", spark.version)
print("sc.version:", spark.sparkContext.version)
try:
    print("scala:", spark._jvm.scala.util.Properties.versionString)
except Exception:
    try:
        print("scala number:", spark._jvm.scala.util.Properties.versionNumberString)
    except Exception as e:
        print("could not read scala version:", e)

pyspark: 3.5.4
spark.version: 3.5.4
sc.version: 3.5.4
scala: <py4j.java_gateway.JavaMember object at 0x10e3739e0>


In [14]:
hosts = ["ac-lvli3cd-shard-00-00.tqttmgs.mongodb.net",
         "ac-lvli3cd-shard-00-01.tqttmgs.mongodb.net",
         "ac-lvli3cd-shard-00-02.tqttmgs.mongodb.net"]
for h in hosts:
    try:
        addr = spark._jvm.java.net.InetAddress.getByName(h)
        print(h, "->", addr.getHostAddress())
    except Exception as e:
        print(h, "DNS resolution failed in JVM:", e)

ac-lvli3cd-shard-00-00.tqttmgs.mongodb.net -> 89.192.8.223
ac-lvli3cd-shard-00-01.tqttmgs.mongodb.net -> 89.192.8.246
ac-lvli3cd-shard-00-02.tqttmgs.mongodb.net -> 89.192.8.236


In [15]:
for h in hosts:
    try:
        s = spark._jvm.java.net.Socket()
        s.connect(spark._jvm.java.net.InetSocketAddress(h, 27017), 5000)  # timeout 5s
        s.close()
        print(h, "TCP connect OK")
    except Exception as e:
        print(h, "TCP connect FAILED in JVM:", e)

ac-lvli3cd-shard-00-00.tqttmgs.mongodb.net TCP connect OK
ac-lvli3cd-shard-00-01.tqttmgs.mongodb.net TCP connect OK
ac-lvli3cd-shard-00-02.tqttmgs.mongodb.net TCP connect OK


In [16]:
props = spark._jvm.java.lang.System.getProperties()
for p in ["http.proxyHost","http.proxyPort","https.proxyHost","https.proxyPort","socksProxyHost","socksProxyPort"]:
    try:
        print(p, "=", props.getProperty(p))
    except Exception:
        pass

http.proxyHost = None
http.proxyPort = None
https.proxyHost = None
https.proxyPort = None
socksProxyHost = None
socksProxyPort = None


In [17]:
try:
    spark._jvm.java.lang.Class.forName("com.mongodb.spark.sql.connector.MongoTableProvider")
    print("MongoTableProvider is available")
except Exception as e:
    print("MongoTableProvider NOT available:", e)

MongoTableProvider NOT available: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: com.mongodb.spark.sql.connector.MongoTableProvider
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:315)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine